## OpenDict DEMO

### 1. Install client libraries

In [1]:
%pip install pyspark-opendic==0.4.0
%pip install snowflake-opendic==0.1.21


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 2 Configure spark-iceberg session

#### 2.1 read_secret()

In [2]:
from pyspark.sql import SparkSession  # type: ignore

def read_secret(secret_name):
    """ Get `secret_name` from docker-compose secret store"""
    secret_path = f"/run/secrets/{secret_name}"
    try:
        with open(secret_path, "r") as f:
            return f.read().strip()  # Remove any trailing newline
    except FileNotFoundError:
        print(f"Secret {secret_name} not found.")
        return None

#### 2.2 Define session variables

In [3]:
CATALOG_NAME = 'AZURE_CATALOG'
ENGINEER_CLIENT_ID = read_secret("engineer_client_id")
ENGINEER_CLIENT_SECRET =  read_secret("engineer_client_secret")
ADLS_IO="org.apache.iceberg.azure.adlsv2.ADLSFileIO"
CATALOG_URI="https://opendict.duckdns.org/api/catalog"


#### 2.3 Configure spark session with variables

In [4]:
def create_session(client_id, client_secret, scope, fileio_impl):
    spark = (SparkSession.builder
        .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.0,software.amazon.awssdk:bundle:2.28.17,software.amazon.awssdk:url-connection-client:2.28.17")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
        .config("spark.sql.catalog.polaris", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.polaris.type", "rest")
        .config("spark.sql.catalog.polaris.warehouse", CATALOG_NAME)
        .config("spark.sql.catalog.polaris.uri", CATALOG_URI)
        .config("spark.sql.catalog.polaris.credential", f"{client_id}:{client_secret}")
        .config("spark.sql.catalog.polaris.scope", scope)
        .config("spark.sql.catalog.my_iceberg.auth.type", "OAUTH2")
        .config("spark.sql.defaultCatalog", "polaris")
        .config("oauth2-server-uri","https://opendict.duckdns.org/api/catalog/v1/oauth/tokens")
        .config("spark.driver.extraJavaOptions", "-Divy.cache.dir=/tmp -Divy.home=/tmp")
        .config("spark.sql.catalog.polaris.token-refresh-enabled", "true")
        .config("spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation", 'vended-credentials')
        .config("spark.sql.catalog.polaris.io-impl", fileio_impl)
        .config("spark.history.fs.logDirectory", "/home/iceberg/spark-events")).getOrCreate()
        
    print("Spark Running")
    return spark


## Start Spark Session
spark = create_session(client_id=ENGINEER_CLIENT_ID, client_secret=ENGINEER_CLIENT_SECRET, scope='PRINCIPAL_ROLE:ALL',fileio_impl=ADLS_IO )
spark

Spark Running


25/06/11 09:29:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


#### 2.4 Wrap spark session with the opendict-spark library

In [8]:
from pyspark_opendic.catalog import OpenDicCatalog  # type: ignore

# Init opendict client library
API_URI= "https://opendict.duckdns.org/api"
catalog = OpenDicCatalog(spark, API_URI)
print("Catalog initialized")

Catalog initialized


### 3. Configure up opendict-snowflake

In [12]:
from snowflake_opendic.snow_opendic import snowflake_connect # type: ignore

def read_secret(secret_name):
    """ Get `secret_name` from docker-compose secret store"""
    secret_path = f"/run/secrets/{secret_name}"
    try:
        with open(secret_path, "r") as f:
            return f.read().strip()  # Remove any trailing newline
    except FileNotFoundError:
        print(f"Secret {secret_name} not found.")
        return None

def snowflake_init_db(conn):
    with conn.cursor() as curr:
        curr.execute("CREATE DATABASE IF NOT EXISTS OPENDIC;")
        curr.execute("use OPENDIC;")
        curr.execute("CREATE SCHEMA IF NOT EXISTS EXPERIMENT;")

ENGINEER_CLIENT_ID = read_secret("engineer_client_id")
ENGINEER_CLIENT_SECRET = read_secret("engineer_client_secret")

print("Secrets read ✔️")

config_path = f"/run/secrets/snowflake-conf"
SNOWFLAKE_CONN = snowflake_connect(config_path)
snowflake_init_db(SNOWFLAKE_CONN)

print("Snowflake conn initialized ✔️")


from snowflake_opendic.catalog import OpenDicSnowflakeCatalog


POLARIS_URI= "https://opendict.duckdns.org/api"

snowflake_catalog = OpenDicSnowflakeCatalog(SNOWFLAKE_CONN, POLARIS_URI, ENGINEER_CLIENT_ID, ENGINEER_CLIENT_SECRET)
print("Catalog initialized ✔️")

Secrets read ✔️


/usr/local/lib/python3.10/site-packages/snowflake_opendic/snow_opendic.py:51: RuntimeWarning: Config file must be in TOML format
  warnings.warn("Config file must be in TOML format", RuntimeWarning)


Snowflake conn initialized ✔️
Connection Established | Server: America/Los_Angeles | Latency: 0.233425 ✔︎
Catalog initialized ✔️


### 4. Scenario and datalake overview

#### 4.1 The AZURE DATALAKE


```
warehouse/
├── SYSTEM/
└── nyc/taxis
```

In [16]:
spark.sql("SHOW NAMESPACES").toPandas()

,namespace
0,SYSTEM
1,nyc


#### 4.2 The taxis dataset

In [15]:
spark.sql("""
          SELECT tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, usd_to_dkk(fare_amount) 
          FROM nyc.taxis limit 10
          """).toPandas()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,usd_to_dkk(fare_amount)
0,2021-04-01 00:00:18,2021-04-01 00:21:54,1.0,8.40,207.825
1,2021-04-01 00:42:37,2021-04-01 00:46:23,1.0,0.90,40.750
2,2021-04-01 00:57:56,2021-04-01 01:08:22,1.0,3.40,93.725
3,2021-04-01 00:01:58,2021-04-01 00:54:27,1.0,0.00,360.230
4,2021-04-01 00:24:55,2021-04-01 00:34:33,1.0,1.96,73.350
5,2021-04-01 00:19:16,2021-04-01 00:21:46,1.0,0.77,36.675
6,2021-04-01 00:25:11,2021-04-01 00:31:53,1.0,3.65,93.725
7,2021-04-01 00:27:53,2021-04-01 00:47:03,0.0,8.90,215.975
8,2021-04-01 00:24:24,2021-04-01 00:37:50,1.0,2.98,97.800
9,2021-04-01 00:19:18,2021-04-01 00:41:25,1.0,8.90,228.200


#### 4.3 Task definition

Translate fare_amount DKK and add a 25% MOMS rate.

**Problem**
- Access same table in Snowflake and Spark
- Want to define once
- Want updates to propogate

**Solution**: 
- OpenDict user-defined object
- Define, create, map, and sync function between multiple engines.

In [ ]:
-- Code example
CREATE FUNCTION usd_to_dkk(amount FLOAT)
  RETURNS FLOAT
  AS
  $$
    amount * 6.52 * 1.25 
  $$

#### 4.4. Define the schema for a OpenDict function object

In [18]:
catalog.sql(
    """
    DEFINE OPEN function
    props {
        "args": "map",
        "language": "string",
        "def": "string",
        "comment": "string",
        "return_type": "string"
    }
    """
)

,udoType,properties,createdTimestamp,lastUpdatedTimestamp,version
0,function,"{'args': 'STRING', 'return_type': 'STRING', 'created_time': 'STRING', 'entity_version': 'STRING', 'uname': 'STRING', 'def': 'STRING', 'last_updated_time': 'STRING', 'language': 'STRING', 'comment': 'STRING'}",1970-01-01T00:00Z,1970-01-01T00:00Z,None


#### 4.5. Create a new function

In [19]:
catalog.sql(
 """
 CREATE OPEN function usd_to_dkk
    PROPS {
            "args": {
                "amount": "DOUBLE"
                },
            "language": "SQL",
            "def": "amount * 6.52 * 1.25",
            "comment": "Conversion function for USD to DKK including VAT",
            "return_type": "DOUBLE"
        }
"""
)

,type-name,object-name,props,created-time-stamp,last-updated-time-stamp,entity-version
0,function,usd_to_dkk,"{'args': {'amount': 'DOUBLE'}, 'return_type': 'DOUBLE', 'language': 'SQL', 'comment': 'Conversion function for USD to DKK including VAT', 'def': 'amount * 6.52 * 1.25'}",2025-06-11T00:44:24.368587804Z,2025-06-11T00:44:24.368588964Z,1


#### 4.6. Create Mappings for spark and snowflake

In [24]:
catalog.sql(
"""
ADD OPEN MAPPING function PLATFORM spark
SYNTAX {
    CREATE <type> <name>(<args>)
    RETURNS <return_type>
    LANGUAGE <language>
    AS 'RETURN <def>';
}
PROPS {
    "args": {
            "propType": "map",
            "format": "<key> <value>",
            "delimiter": ", "
        }
}
"""
)

,typeName,platformName,syntax,objectDumpMap,createdTimestamp,lastUpdatedTimestamp,version
0,function,spark,CREATE <type> <name>(<args>)\n RETURNS <return_type>\n LANGUAGE <language>\n AS 'RETURN <def>';,"{'args': {'propType': 'map', 'format': '<key> <value>', 'delimiter': ', '}}",2025-06-11T00:49:36.726836830Z,2025-06-11T00:49:36.726837190Z,1


In [ ]:
catalog.sql(
"""
ADD OPEN MAPPING function PLATFORM snowflake
SYNTAX {
CREATE OR REPLACE <type> <name>(<args>)
RETURNS <return_type>
LANGUAGE <language>
AS 
$$
<def>
$$;
}
PROPS {
    "args": {
            "propType": "map",
            "format": "<key> <value>",
            "delimiter": ", "
        }
}
"""
)

,typeName,platformName,syntax,objectDumpMap,createdTimestamp,lastUpdatedTimestamp,version
0,function,snowflake,CREATE OR REPLACE <type> <name>(<args>)\nRETURNS <return_type>\nLANGUAGE <language>\nAS \n$$\n<def>\n$$;,"{'args': {'propType': 'map', 'format': '<key> <value>', 'delimiter': ', '}}",2025-06-11T00:51:46.053997346Z,2025-06-11T00:51:46.053997746Z,1


#### 4.7 Sync to engines

In [11]:
catalog.sql(
    """
    SYNC OPEN OBJECTS for spark
    """
)

25/06/11 09:30:34 WARN SimpleFunctionRegistry: The function usd_to_dkk replaced a previously registered function.


```json
{
    "executions": [
        {
            "sql": "CREATE function usd_to_dkk(amount DOUBLE)\n    RETURNS DOUBLE\n    LANGUAGE SQL\n    AS 'RETURN amount * 6.52 * 1.25';",
            "status": "function registered"
        }
    ]
}
```

In [10]:
snowflake_catalog.sql(
    """
    SYNC OPEN OBJECTS for snowflake
    """
)

NameError: name 'snowflake_catalog' is not defined

### Utils

#### List objects

In [ ]:
catalog.sql(
    """
    SHOW OPEN TYPES
    """
)

In [ ]:
catalog.sql(
    """
    SHOW OPEN function
    """
)

In [ ]:
# Show mapping for <object> to <platform>. Example: [Platform_mapping(function_v2 -> snowflake)]
catalog.sql(
    """
    SHOW OPEN MAPPING function_v2 PLATFORM snowflake
    """
)

In [ ]:
# Show all mappings from <object>. Example: [snowflake,spark]
catalog.sql(
    """
    SHOW OPEN PLATFORMS FOR function_v2
    """
)

In [ ]:
catalog.sql(
    """
    SHOW OPEN PLATFORMS
    """
)

In [ ]:
catalog.sql(
    """
    SHOW OPEN MAPPINGS FOR snowflake
    """
)

In [ ]:
catalog.sql(
    """
    SYNC OPEN function_v2 for snowflake
    """
)

In [ ]:
catalog.sql(
    """
    SYNC OPEN OBJECTS for snowflake
    """
)

#### Drop objects

In [ ]:
catalog.sql(
    """
    DROP OPEN function
    """
)


In [22]:
catalog.sql(
    """
    DROP OPEN MAPPINGS for snowflake
    """
)


```json
{
    "error": "HTTP Error",
    "details": "404 Client Error: Not Found for url: https://opendict.duckdns.org/api/opendic/v1/platforms/snowflake",
    "Catalog Response": null
}
```

In [23]:
catalog.sql(
    """
    DROP OPEN MAPPINGS for spark
    """
)

,Deleted all mappings for platform
0,spark


#### Visualize opendic tables

In [ ]:
%%sql
show tables in SYSTEM

In [ ]:
%%sql
select uname as name, args, return_type, signature, runtime, language, comment, packages, def   from SYSTEM.function_v2

In [ ]:
%%sql
USE SYSTEM

In [ ]:
%%sql
select * from SYSTEM.function

In [ ]:
%%sql

CREATE DATABASE IF NOT EXISTS nyc

In [ ]:
%%sql

DROP TABLE IF EXISTS nyc.taxis

In [ ]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df.write.saveAsTable("nyc.taxis")

In [ ]:
%%sql

SELECT *
FROM nyc.taxis limit 10